In [1]:
import requests
import json
import datetime
import pandas as pd

In [2]:
claimed_corgis = pd.read_csv('claimed_corgis.csv')
claimed_blocks = list(claimed_corgis.block)

In [3]:
corgi_dict = {'Number': [], 'Block': [],
              'Base': [], 'Fluff': [], 'Forehead': [], 'Belly': [],
              'Mood': [], 'Nose': [], 'Tail': [], 'Pelt': [],
              'Outline': [], 'Paws': [], 'Head': [], 'Face': [],
              'Bottoms': [], 'Top': [], 'Tools': [], 'Unique': [],
              'Status': [], 'Price': [], 'Offer': [], 'MaxOffer': []}

In [4]:
def crawlOpenSea(block_list):
    '''
    crawlOpenSea requests the OpenSea API for every block number in block_list,
    specifically for the Crypto Corgis contract address
    '''
    
    print('START: ' + str(datetime.datetime.now()))
    
    count = 0
    for block in block_list:

        # Progress check
        count += 1
        if count % 100 == 0:
            print(str(count) + ': ' + str(datetime.datetime.now()))

        # OpenSea API, retry when throttled
        throttle = True
        while throttle == True:
            url = 'https://api.opensea.io/api/v1/asset/0x51e613727fdd2e0b91b51c3e5427e9440a7957e4/' + str(block)
            response = requests.request("GET", url)
            asset = json.loads(response.text)

            if len(asset) > 10:
                throttle = False

        # Proceed if block is associated with a living Corgi
        if (asset['name'] != None) and (asset['name'][0:7] == 'Corgi #'):

            if block not in claimed_blocks:
                claimed_blocks.append(block)

            corgi_dict['Number'].append(int(asset['name'][7:]))
            corgi_dict['Block'].append(block)

            # Populate trait columns
            traits = ['Base', 'Fluff', 'Forehead', 'Belly', 'Mood', 'Nose', 'Tail', 'Pelt', 'Outline', 'Paws', 'Head', 'Face', 'Bottoms', 'Top', 'Tools', 'Unique']
            for trait in asset['traits']:
                corgi_dict[trait['trait_type']].append(trait['value'])
                traits.remove(trait['trait_type'])

            for trait in traits:
                corgi_dict[trait].append('NA')

            # Populate order and price columns
            offers = []
            listed = False

            for order in asset['orders']:

                # Listed for sale or auction
                if order['side'] == 1:
                    listed = True
                    if order['payment_token_contract']['id'] == 1:
                        corgi_dict['Status'].append('for_sale')
                        corgi_dict['Price'].append(float(order['base_price'])/1e18)
                    elif order['payment_token_contract']['id'] == 2:
                        corgi_dict['Status'].append('auction')
                        corgi_dict['Price'].append(float(order['base_price'])/1e18)

                # Offers
                elif order['side'] == 0:
                    offers.append(float(order['base_price'])/1e18)

            # Not listed for sale or auction
            if listed == False:
                corgi_dict['Status'].append('unlisted')
                corgi_dict['Price'].append(0.0)

            # No offers
            if offers == []:
                corgi_dict['Offer'].append(0)
                corgi_dict['MaxOffer'].append(0.0)

            # Highest offer
            else:
                corgi_dict['Offer'].append(1)
                corgi_dict['MaxOffer'].append(max(offers))

    print('END: ' + str(datetime.datetime.now()))

In [5]:
crawlOpenSea(claimed_blocks)

START: 2021-03-28 12:02:26.629566
100: 2021-03-28 12:03:19.918080
200: 2021-03-28 12:04:12.845214
300: 2021-03-28 12:05:06.763323
400: 2021-03-28 12:06:01.130736
500: 2021-03-28 12:06:55.330658
600: 2021-03-28 12:07:48.714520
700: 2021-03-28 12:08:45.475795
800: 2021-03-28 12:09:42.669841
900: 2021-03-28 12:10:35.605822
1000: 2021-03-28 12:11:28.509818
1100: 2021-03-28 12:12:22.774390
1200: 2021-03-28 12:13:20.715848
1300: 2021-03-28 12:14:14.195753
1400: 2021-03-28 12:15:07.590856
1500: 2021-03-28 12:16:00.482289
1600: 2021-03-28 12:16:55.836125
1700: 2021-03-28 12:17:48.570251
1800: 2021-03-28 12:18:43.189135
1900: 2021-03-28 12:19:37.945875
2000: 2021-03-28 12:20:33.410847
2100: 2021-03-28 12:21:30.521444
2200: 2021-03-28 12:22:24.423887
2300: 2021-03-28 12:23:19.328753
2400: 2021-03-28 12:24:12.429429
2500: 2021-03-28 12:25:06.302219
2600: 2021-03-28 12:26:01.583256
2700: 2021-03-28 12:26:54.664782
2800: 2021-03-28 12:27:49.407157
2900: 2021-03-28 12:28:43.023872
END: 2021-03-28 12

In [6]:
url = 'https://api.blockcypher.com/v1/eth/main'
response = requests.request("GET", url)
dict = json.loads(response.text)

# first_block = 12080536
last_block = dict['height'] # last Ethereum block
pending_blocks = 256 # Corgis die after being unclaimed for 256 blocks

print('Last Block: ' + str(last_block))

unclaimed_blocks = list(range(max(claimed_blocks) - pending_blocks, last_block + 1))

for block in unclaimed_blocks:
    if block in claimed_blocks:
        unclaimed_blocks.remove(block)

for block in claimed_blocks:
    if block not in corgi_dict['Block']:
        unclaimed_blocks.append(block)

Last Block: 12128717


In [7]:
crawlOpenSea(unclaimed_blocks)

START: 2021-03-28 12:29:16.262351
100: 2021-03-28 12:30:08.664978
200: 2021-03-28 12:31:05.979238
300: 2021-03-28 12:32:00.639289
400: 2021-03-28 12:32:55.587430
500: 2021-03-28 12:33:49.789242
600: 2021-03-28 12:34:44.165726
700: 2021-03-28 12:35:36.834919
800: 2021-03-28 12:36:32.591896
900: 2021-03-28 12:37:25.754962
1000: 2021-03-28 12:38:41.407958
1100: 2021-03-28 12:40:24.960286
1200: 2021-03-28 12:42:08.100263
1300: 2021-03-28 12:43:38.483402
1400: 2021-03-28 12:45:17.795277
1500: 2021-03-28 12:46:52.649371
1600: 2021-03-28 12:48:31.924495
END: 2021-03-28 12:49:50.502101


In [8]:
claimed_corgis = pd.DataFrame({'block': claimed_blocks})
claimed_corgis.to_csv('claimed_corgis.csv', index = False)

In [9]:
corgi_df = pd.DataFrame(corgi_dict)
corgi_df.head()

,Number,Block,Base,Fluff,Forehead,Belly,Mood,Nose,Tail,Pelt,...,Head,Face,Bottoms,Top,Tools,Unique,Status,Price,Offer,MaxOffer
0,1,12080536,Classic Plus,NA,NA,NA,Happy,NA,NA,NA,...,Dark Pink Sweatband,3D Glasses,NA,NA,Lemon Icepop,NA,unlisted,0.0,1,0.2
1,2,12080767,Classic,NA,NA,NA,Pleased,NA,Light Brown,NA,...,NA,Red Glasses,Brown,NA,Water,NA,unlisted,0.0,0,0.0
2,3,12080747,Classic,NA,NA,Off White,Happy,Very Cold,NA,NA,...,Dark Pink Sweatband,Red Glasses,NA,NA,Blueberry Icepop,NA,unlisted,0.0,0,0.0
3,4,12082041,Black,NA,NA,Brown,Pleased,Grey,Blue,NA,...,Grey Crown,Blue Round Shades,Dark Green,NA,NA,NA,for_sale,5.0,1,0.1
4,5,12081917,Classic,NA,Blue,NA,Pleased,NA,Tan,NA,...,Dark Pink Sweatband,Surprised Brows,Dark Green,NA,Dumbbell,NA,unlisted,0.0,0,0.0


In [10]:
corgi_df.to_csv('crypto_corgis_df.csv', index = False)

# Visualization

In [11]:
from pandasgui import show

In [12]:
corgi_df = pd.read_csv('crypto_corgis_df.csv')
corgi_df.head()

,Number,Block,Base,Fluff,Forehead,Belly,Mood,Nose,Tail,Pelt,...,Head,Face,Bottoms,Top,Tools,Unique,Status,Price,Offer,MaxOffer
0,1,12080536,Classic Plus,NaN,NaN,NaN,Happy,NaN,NaN,NaN,...,Dark Pink Sweatband,3D Glasses,NaN,NaN,Lemon Icepop,NaN,unlisted,0.0,1,0.2
1,2,12080767,Classic,NaN,NaN,NaN,Pleased,NaN,Light Brown,NaN,...,NaN,Red Glasses,Brown,NaN,Water,NaN,unlisted,0.0,0,0.0
2,3,12080747,Classic,NaN,NaN,Off White,Happy,Very Cold,NaN,NaN,...,Dark Pink Sweatband,Red Glasses,NaN,NaN,Blueberry Icepop,NaN,unlisted,0.0,0,0.0
3,4,12082041,Black,NaN,NaN,Brown,Pleased,Grey,Blue,NaN,...,Grey Crown,Blue Round Shades,Dark Green,NaN,NaN,NaN,for_sale,5.0,1,0.1
4,5,12081917,Classic,NaN,Blue,NaN,Pleased,NaN,Tan,NaN,...,Dark Pink Sweatband,Surprised Brows,Dark Green,NaN,Dumbbell,NaN,unlisted,0.0,0,0.0


In [13]:
show(corgi_df)